In [1]:
dataset_this_month = '26 Feb to 20 Mar.xlsx'

# Importing Data And Libraries

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from apyori import apriori

### Reading Our Datasets 

In [3]:
dataset = pd.read_excel(dataset_this_month)
df_all_months = dataset.copy()

# Cleaning Part

### Extract the year and the month columns


In [6]:
df_all_months['day'] = pd.to_datetime(df_all_months['Date']).dt.day
d = list(df_all_months['day'].dropna().unique())

df_all_months['month'] = pd.to_datetime(df_all_months['Date']).dt.month_name()
m = list(df_all_months['month'].dropna().unique())

df_all_months['year'] = pd.to_datetime(df_all_months['Date']).dt.year
y = list(df_all_months['year'].dropna().unique())

In [18]:
#df['month_day'] = str(df_all_months['day']) + (df_all_months['month'])
df_all_months['date'] = pd.to_datetime(df_all_months['Date']).dt.date
x = list(df_all_months['date'])
x

[datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date(2024, 2, 26),
 datetime.date

In [ ]:
[]

In [16]:
this_month = df_all_months[df_all_months['date']== '2024-02-26']


### Change the directory in order to put all the excel files reports in the day folder

In [10]:
root_path = 'Desktop/'
os.mkdir(os.path.join(root_path, ('from '+  str(d[0]) + ' ' + str(m[0]) + ' to ' + str(m[-1]) + ' '+ str(d[-1]) + ' ' +str(int(y[0])))))
os.chdir(os.path.join(root_path,  ('from '+  str(d[0]) + ' ' + str(m[0]) + ' to ' + str(m[-1]) + ' '+ str(d[-1]) + ' ' +str(int(y[0])))))

### Split product cat Columns

In [11]:
try:
    df[['Product origin', 'Cat1', 'cat2', 'cat3']]= df['Product Category'].str.split("/",expand=True)
    df.drop(columns= {'Product Category','Unnamed: 14' }, inplace = True, errors = 'ignore')
except:
    df[['Product origin', 'Cat1', 'cat2', 'cat3']]= df['Product Category'].str.split("/",expand=True)
    df.drop(columns= {'Product Category','Unnamed: 14' }, inplace = True, errors = 'ignore')

try:
    df_pervious[['Product origin', 'Cat1', 'cat2', 'cat3', 'Unnamed: 14']]= df_pervious['Product Category'].str.split("/",expand=True)
    df_pervious.drop(columns= {'Product Category'}, inplace = True)
except: 
    df_pervious[['Product origin', 'Cat1', 'cat2', 'cat3']]= df_pervious['Product Category'].str.split("/",expand=True)
    df_pervious.drop(columns= {'Product Category'}, inplace = True)
    
try:
    dataset_pervious_monthyear[['Product origin', 'Cat1', 'cat2', 'cat3', 'Unnamed: 14']]= dataset_pervious_monthyear['Product Category'].str.split("/",expand=True)
    dataset_pervious_monthyear.drop(columns= {'Product Category'}, inplace = True)
except: 
    dataset_pervious_monthyear[['Product origin', 'Cat1', 'cat2', 'cat3']]= dataset_pervious_monthyear['Product Category'].str.split("/",expand=True)
    dataset_pervious_monthyear.drop(columns= {'Product Category'}, inplace = True)

### Prepare the required Columns

In [12]:
df['discount amount'] = df['Quantity'] * df['Discount per Unit (abs)']
df['discount percentage'] = ((df['Discount (%)'] / 100) * df['Unit Price']) *df['Quantity']
df['Total discount'] = df['discount amount']  + df['discount percentage']
df['price before discount'] = df['Unit Price'] * df['Quantity']
df['Cost per unit'] = df['Cost'] * df['Quantity']
df['vat'] = df['Subtotal'] / 1.15
df['profit margin'] = df['vat'] - df['Cost'] 
df['Profit margin percentage'] = df['profit margin'] /df['vat']

In [13]:
df_pervious['discount amount'] = df_pervious['Quantity'] * df_pervious['Discount per Unit (abs)']
df_pervious['discount percentage'] = ((df_pervious['Discount (%)'] / 100) * df_pervious['Unit Price']) *df_pervious['Quantity']
df_pervious['Total discount'] = df_pervious['discount amount']  + df_pervious['discount percentage']
df_pervious['price before discount'] = df_pervious['Unit Price'] * df_pervious['Quantity']
df_pervious['Cost per unit'] = df_pervious['Cost'] * df_pervious['Quantity']
df_pervious['vat'] = df_pervious['Subtotal'] / 1.15
df_pervious['profit margin'] = df_pervious['vat'] - df_pervious['Cost'] 
df_pervious['Profit margin percentage'] = df_pervious['profit margin'] /df_pervious['vat']

In [14]:
dataset_pervious_monthyear['discount amount'] = dataset_pervious_monthyear['Quantity'] * dataset_pervious_monthyear['Discount per Unit (abs)']
dataset_pervious_monthyear['discount percentage'] = ((dataset_pervious_monthyear['Discount (%)'] / 100) * dataset_pervious_monthyear['Unit Price']) *dataset_pervious_monthyear['Quantity']
dataset_pervious_monthyear['Total discount'] = dataset_pervious_monthyear['discount amount']  + dataset_pervious_monthyear['discount percentage']
dataset_pervious_monthyear['price before discount'] = dataset_pervious_monthyear['Unit Price'] * dataset_pervious_monthyear['Quantity']
dataset_pervious_monthyear['Cost per unit'] = dataset_pervious_monthyear['Cost'] * dataset_pervious_monthyear['Quantity']
dataset_pervious_monthyear['vat'] = dataset_pervious_monthyear['Subtotal'] / 1.15
dataset_pervious_monthyear['profit margin'] = dataset_pervious_monthyear['vat'] - dataset_pervious_monthyear['Cost'] 
dataset_pervious_monthyear['Profit margin percentage'] = dataset_pervious_monthyear['profit margin'] /dataset_pervious_monthyear['vat']

### Cleaning the values of the branches to be in a better shape

In [15]:
df['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : 'الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : 'بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

In [16]:
df_pervious['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : 'الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : 'بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

In [17]:
dataset_pervious_monthyear['فريق المبيعات'].replace({ 'مبيعات نقاط بيع راحتي الدمام': 'الدمام'
                             ,'مبيعات نقاط بيع فرع بريدة الكبير 1' : 'بريدة الكبير 1', 
                            'مبيعات نقاط بيع راحتي الرس': 'الرس',
                            'مبيعات نقاط  بيع راحتي  الرياض 1 اثاث': 'الرياض 1 اثاث',
                             'مبيعات نقاط بيع راحتي حائل 1' : 'حائل 1',
                            'مبيعات نقاط بيع راحتي الطائف الكبير 1' : 'الطائف الكبير 1', 
                            'مبيعات نقاط بيع راحتي بريدة الكبير 2' : 'بريدة الكبير 2',
                            'مبيعات نقاط بيع قصر البيان حائل': 'قصر البيان حائل', 
                            'مبيعات نقاط بيع راحتي حفر الباطن': 'حفر الباطن',
                            'مبيعات نقاط بيع راحتي حائل 2' : 'حائل 2', 
                            'مبيعات نقاط بيع راحتي البدائع': 'البدائع',
                             'مبيعات نقاط بيع متجر راحتي الالكتروني': 'متجر راحتي الالكتروني',
                             'مبيعات نقاط بيع راحتي الرياض2 مفروشات': 'الرياض2 مفروشات',
                            'مبيعات نقاط بيع راحتي الطائف الصغير 2' : 'الطائف الصغير 2',
                             'مبيعات نقاط بيع قصر البيان الرس': 'قصر البيان الرس',
                             'مبيعات متجر قصر البيان الإلكتروني': 'قصر البيان الإلكتروني', 
                             'مبيعات فرع تبوك': 'فرع تبوك',
                            'مبيعات الموقع تبوك': 'الموقع تبوك'}, inplace = True)

### Prepare different pages 

#### Page of Deposits

In [18]:
df_deposit = df[df['Product origin'] == 'منتجات مصروفات الشركة']
df_deposit.to_excel('Service and other things report.xlsx', sheet_name='منتجات مصروفات الشركة',  index = False)
df.loc[df_deposit.index,"Quantity"] = 1

#### Page of services and installment

In [19]:
df_services = df[df['Product origin'] == 'خدمات-44 ']
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_services.to_excel(writer, sheet_name= 'خدمات-44',  index = False)


In [20]:
df_installment = df[df['Product origin'] == 'خدمات التقسيط']
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_installment.to_excel(writer, sheet_name= 'خدمات التقسيط',  index = False)
df_services =pd.concat([df_services, df_installment])

#### Get the additional discounts rows, put it in dataframe and save it as a file

In [21]:
df_all = df[df['Product origin'] == 'All']
df_addition_discount = df_all[df_all['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all.drop(df_addition_discount.index, inplace = True)
df_services = pd.concat([df_services, df_all])
df_addition_discount.to_excel('addition_discount report.xlsx',  index = False)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_14196\3686595714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all.drop(df_addition_discount.index, inplace = True)


In [22]:
df_all_pervious = df_pervious[df_pervious['Product origin'] == 'All']
df_addition_discount_pervious = df_pervious[df_pervious['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all_pervious.drop(df_addition_discount_pervious.index, inplace = True)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_14196\111177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_pervious.drop(df_addition_discount_pervious.index, inplace = True)


In [23]:
df_all_monthyear = dataset_pervious_monthyear[dataset_pervious_monthyear['Product origin'] == 'All']
df_addition_discount_pervious = dataset_pervious_monthyear[dataset_pervious_monthyear['Product'] == '[11223344] RS خصم اضافي للعميل']
df_all_monthyear.drop(df_addition_discount_pervious.index, inplace = True)

C:\Users\Rahaty Store\AppData\Local\Temp\ipykernel_14196\709480280.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_all_monthyear.drop(df_addition_discount_pervious.index, inplace = True)


#### Page of zero qunatity

In [24]:
df_0 = df[df['Quantity'] == 0 ]
with pd.ExcelWriter('Service and other things report.xlsx', engine='openpyxl', mode='a') as writer:
    df_0.to_excel(writer, sheet_name= 'صفر كمية',  index = False)
df.drop(df_0.index, inplace = True)

In [25]:
df_0 = df_pervious[df_pervious['Quantity'] == 0 ]
df_pervious.drop(df_0.index, inplace = True)

In [26]:
df_0 = dataset_pervious_monthyear[dataset_pervious_monthyear['Quantity'] == 0 ]
dataset_pervious_monthyear.drop(df_0.index, inplace = True)

#### Final Total

In [27]:
df_final_total = df[df['Product'] == 'الإجمالي النهائي' ]
df.drop(df_final_total.index, inplace = True)


In [28]:
df_final_total = df_pervious[df_pervious['Product'] == 'الإجمالي النهائي' ]
df_pervious.drop(df_final_total.index, inplace = True)

In [29]:
df_final_total = dataset_pervious_monthyear[dataset_pervious_monthyear['Product'] == 'الإجمالي النهائي' ]
dataset_pervious_monthyear.drop(df_final_total.index, inplace = True)

# Analytical Part

## Sales Report For Branches 

### Sales Worksheet

In [30]:
# Getting the sales of each branch in the company
df_sales= pd.pivot_table(df,index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
df_sales.rename(columns ={'Subtotal':'total sales'}, inplace = True)

# Getting the branch contribution of the sales of the company
df_sales['Sales Percent from the company'] = df_sales['total sales']/ df_sales['total sales'].sum()
df_sales.rename(columns = {'total sales': "إجمالي المبيعات"
                           , 'Sales Percent from the company': 'نسبة الفرع من إجمالي المبيعات' },inplace = True)

# Getting the sales of each branch in the company in the previous month
df_pervious_sales = pd.pivot_table(df_pervious,index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
df_pervious_sales.rename(columns ={'Subtotal':'إجمالي مبيعات الشهر السابق'}, inplace = True)


# Merging both the sales of the day and the sales of the previous month
df_sales = df_sales.merge(df_pervious_sales, on ='فريق المبيعات', how = 'left')


# getting the change of sales for each branch
df_sales['معدل التغير عن الشهر السابق'] = (df_sales['إجمالي المبيعات'] - df_sales['إجمالي مبيعات الشهر السابق']) / df_sales['إجمالي مبيعات الشهر السابق']
df_sales.drop(columns = {'إجمالي مبيعات الشهر السابق'}, inplace = True)

# filling the nan values for the branches that didn't sale anything or the branches that didn't exist in the previous Month
df_sales.fillna(0, inplace =  True)


# Getting the sales of each branch in the company in this month in the pervious year
df_pervious_year_sales = pd.pivot_table(dataset_pervious_monthyear,index=['فريق المبيعات'], values=(['Subtotal']), aggfunc = 'sum')
df_pervious_year_sales.rename(columns ={'Subtotal':'إجمالي مبيعات الشهر الحالي للسنة السابقة'}, inplace = True)

# Merging both the sales of the day and the sales of this month in the pervious year
df_sales = df_sales.merge(df_pervious_year_sales, on ='فريق المبيعات', how = 'left')


# getting the change of sales for each branch
df_sales['معدل التغير عن الشهر الحالي للسنة السابقة'] = (df_sales['إجمالي المبيعات'] - df_sales['إجمالي مبيعات الشهر الحالي للسنة السابقة']) / df_sales['إجمالي مبيعات الشهر الحالي للسنة السابقة']
df_sales.drop(columns = {'إجمالي مبيعات الشهر الحالي للسنة السابقة'}, inplace = True)


# filling the nan values for the branches that didn't sale anything or the branches that didn't exist in the previous Month
df_sales.fillna(0, inplace =  True)


# sorting the branches based on their performance
df_sales = df_sales.sort_values(by= ['إجمالي المبيعات'], ascending = False)
df_sales.reset_index(inplace = True)


# encoding the last column in the dataset and summarize the results of all branches 
df_sales_final = df_sales.copy()
df_sales_final.loc['الاجمالي', 'فريق المبيعات'] =  df_sales['فريق المبيعات'].count()
df_sales_final.loc['الاجمالي', 'معدل التغير عن الشهر السابق'] =(df_sales['إجمالي المبيعات'].sum() - df_pervious_sales['إجمالي مبيعات الشهر السابق'].sum())/ df_pervious_sales['إجمالي مبيعات الشهر السابق'].sum()
df_sales_final.loc['الاجمالي', 'معدل التغير عن الشهر الحالي للسنة السابقة'] =(df_sales['إجمالي المبيعات'].sum() - df_pervious_year_sales['إجمالي مبيعات الشهر الحالي للسنة السابقة'].sum())/ df_pervious_year_sales['إجمالي مبيعات الشهر الحالي للسنة السابقة'].sum()
df_sales_final.loc['الاجمالي', 'نسبة الفرع من إجمالي المبيعات'] = df_sales['نسبة الفرع من إجمالي المبيعات'].sum()
df_sales_final.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()
df_sales_final.loc['الاجمالي', 'إجمالي المبيعات'] = df_sales['إجمالي المبيعات'].sum()

# getting the order column
df_sales_final.reset_index(inplace = True)
df_sales_final.rename(columns = {'index': 'الترتيب'}, inplace = True)
df_sales_final.set_index('الترتيب', inplace = True)

# saving the insights in the sales report 
df_sales_final.to_excel('sales report.xlsx', sheet_name='مبيعات الفروع')

In [ ]:
# Getting the profit of each branch in the company
df_profit= pd.pivot_table(df.drop(list(df_deposit.index)),index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
df_profit.rename(columns ={'profit margin':'total Profit'}, inplace = True)

# Getting the branch contribution of the profit of the company
df_profit['Profit Percent from the company'] = df_profit['total Profit']/ df_profit['total Profit'].sum()
df_profit.rename(columns = {'total Profit': "إجمالي الربح"
                           , 'Profit Percent from the company': 'نسبة الفرع من إجمالي الأرباح' },inplace = True)

# Getting the vat for each branch
df_margin_p= pd.pivot_table(df.drop(list(df_deposit.index)),index=['فريق المبيعات'], values=(['vat']), aggfunc = 'sum')
df_margin_p.rename(columns ={'vat':'الربح قبل الضريبة'}, inplace = True)

# Adding the vat to the profit dataframe
df_profit = df_profit.merge(df_margin_p, on ='فريق المبيعات', how = 'inner')

# getting the profit margin
df_profit['هامش الربح من المبيعات'] =df_profit['إجمالي الربح'] /  df_profit['الربح قبل الضريبة']  
df_profit.drop(columns = {'الربح قبل الضريبة'}, inplace = True)

# Getting the profit of each branch in the company in the previous day
df_pervious_profit = pd.pivot_table(df_pervious,index=['فريق المبيعات'], values=(['profit margin']), aggfunc = 'sum')
df_pervious_profit.rename(columns ={'profit margin':'إجمالي ربح اليوم السابق'}, inplace = True)

# Merging both the profit of the day and the profit of the previous day
df_profit = df_profit.merge(df_pervious_profit, on ='فريق المبيعات', how = 'left')

# getting the change of profit for each branch
df_profit['معدل التغير عن اليوم السابق'] = (df_profit['إجمالي الربح'] - df_profit['إجمالي ربح اليوم السابق']) / df_profit['إجمالي ربح اليوم السابق']
df_profit.drop(columns = {'إجمالي ربح اليوم السابق'}, inplace = True)

# filling the nan values for the branches that didn't sale anything
df_profit.fillna(0, inplace =  True)

# sorting the branches based on their performance
df_profit = df_profit.sort_values(by= "إجمالي الربح", ascending = False)
df_profit.reset_index(inplace = True)

# making a copy of the profit data in order to safely make changes 
df_profit_final = df_profit.copy()

# encoding the last column in the dataset and summarize the results of all branches 
df_profit_final.loc['الاجمالي', 'فريق المبيعات'] =  df_profit['فريق المبيعات'].count()
df_profit_final.loc['الاجمالي', 'معدل التغير عن اليوم السابق'] =(df_profit['إجمالي الربح'].sum() - df_pervious_profit['إجمالي ربح اليوم السابق'].sum())/ df_pervious_profit['إجمالي ربح اليوم السابق'].sum()
df_profit_final.loc['الاجمالي', 'هامش الربح من المبيعات'] = df_profit['إجمالي الربح'].sum() / (df_sales['إجمالي المبيعات'].sum()/1.15)
df_profit_final.loc['الاجمالي', 'نسبة الفرع من إجمالي الأرباح'] = df_profit['نسبة الفرع من إجمالي الأرباح'].sum()
df_profit_final.loc['الاجمالي', 'إجمالي الربح'] = df_profit['إجمالي الربح'].sum()

# getting the order column
df_profit_final.reset_index(inplace = True)
df_profit_final.rename(columns = {'index': 'الترتيب'}, inplace = True)
df_profit_final.set_index('الترتيب', inplace = True)

# saving the profit insights in the sales report 
with pd.ExcelWriter('sales report.xlsx', engine='openpyxl', mode='a') as writer:
    df_profit_final.to_excel(writer, sheet_name= 'اكثر الفروع ربحا')

In [ ]:
# Getting the data of branches with profit margins less than 30% 
df_bad_performance = df_profit_final[df_profit_final['هامش الربح من المبيعات'] < .30]

with pd.ExcelWriter('sales report.xlsx', engine='openpyxl', mode='a') as writer:
    df_bad_performance.to_excel(writer, sheet_name= 'اقل هامش ربح')